In [1]:
import pymongo
import pandas as pd
import numpy as np
import json
from pandas.io.json import json_normalize
import warnings
warnings.filterwarnings('ignore')

In [2]:
conn = pymongo.MongoClient('localhost', 27017)

In [3]:
conn.database_names()

['admin', 'config', 'local', 'smartcareer']

In [4]:
conn.smartcareer.collection_names()

['test', 'dataengineer']

In [5]:
db = conn.smartcareer

In [6]:
col = db.dataengineer

In [154]:
data = []

for doc in col.find():
    data.append(doc)

In [155]:
data[9]

{'_id': ObjectId('5dca13fcf68f858820733cab'),
 'ProfileID': 'qihang-yan',
 'Job Title': 'Incoming Data Engineer at Amazon Robotics',
 'Location': 'Ithaca, New York',
 'Profile Summary': '',
 'Experience': [{'Job Title': 'Incoming Data Engineer',
   'Company': 'Amazon Robotics',
   'Period': 'Oct 2019 – Present',
   'Years': '2 mos',
   'Location': 'North Reading, Massachusetts',
   'Description': ''},
  {'Job Title': 'Data Engineer Intern',
   'Company': 'Amazon Robotics',
   'Period': 'May 2019 – Aug 2019',
   'Years': '4 mos',
   'Location': '',
   'Description': '• Designed and implemented a data-driven, knowledge-based framework in Python3 for classifying root causes of any inefficiencies in the Amazon’s warehouse system using K-Means Clustering and dynamic statistical analysis. • Deployed the framework end-to-end in a development pipeline using Native AWS services, including Lambda Functions, ECS, Fargate, Athena, S3 and etc. • Deep dived to identify where inefficiencies could occ

In [156]:
df = pd.DataFrame(data)
df.head()

,Date Captured,Education,Experience,Industry Knowledge,Interpersonal Skills,Job Title,Languages,Location,Other Skills,Profile Summary,ProfileID,Skills & Endorsements,Tools & Technologies,_id
0,2019-11-12,"[{'School': 'University of Washington', 'Degre...","[{'Job Title': 'Senior Data Engineer', 'Compan...","[{'Skills': 'Research'}, {'Skills': 'Data Anal...",NaN,Senior Data Engineer at Change Healthcare,NaN,San Francisco Bay Area,"[{'Skills': 'Apache Pig'}, {'Skills': 'Apache ...",As a software engineer at an research and deve...,jilliankim,"[{'Skills': 'Apache Spark'}, {'Skills': 'Apach...","[{'Skills': 'Java'}, {'Skills': 'HBase'}, {'Sk...",5dca1376f68f858820733c99
1,2019-11-12,"[{'School': 'Korea University', 'Degree': 'Doc...","[{'Job Title': 'Director of Research', 'Compan...","[{'Skills': 'Algorithms'}, {'Skills': 'Big Dat...",NaN,Sr. Principal Data Engineer at Coupang Global ...,NaN,San Francisco Bay Area,"[{'Skills': 'Data Processing'}, {'Skills': 'La...",Specialties: * Query evaluation techniques in ...,hyunsikchoi,"[{'Skills': 'Hadoop'}, {'Skills': 'Distributed...","[{'Skills': 'Open Source'}, {'Skills': 'Java'}...",5dca1380f68f858820733c9b
2,2019-11-12,"[{'School': 'University of Virginia', 'Degree'...","[{'Job Title': 'Data Engineer', 'Company': 'Pl...",NaN,NaN,Data Engineer at PlayStation,NaN,"Los Angeles, California",NaN,,rui-deng-b7b37714a,"[{'Skills': 'Python (Programming Language)'}, ...",NaN,5dca13b2f68f858820733c9d
3,2019-11-12,"[{'School': 'University of California, Berkele...",[{'Job Title': 'Senior Site Reliability Engine...,"[{'Skills': 'Web Applications'}, {'Skills': 'W...",NaN,Architect / Data engineer,NaN,"Berkeley, California","[{'Skills': 'Memcached'}, {'Skills': 'Google A...","Energetic and motivating, those are the two wo...",seungjoonlee1984,"[{'Skills': 'JavaScript'}, {'Skills': 'Python'...","[{'Skills': 'Java'}, {'Skills': 'JSON'}, {'Ski...",5dca13bff68f858820733c9f
4,2019-11-12,"[{'School': 'Flatiron School', 'Degree': ' Dat...","[{'Job Title': 'Data Scientist Student', 'Comp...","[{'Skills': 'Web Development'}, {'Skills': 'An...",[{'Skills': 'Written Communication'}],Data Scientist and ML Engineer,NaN,Greater Seattle Area,"[{'Skills': '모바일앱'}, {'Skills': 'C++ Language'...",Machine learning engineer and data scientist w...,taeho-jeon-78418870,"[{'Skills': 'iOS development'}, {'Skills': 'Ja...","[{'Skills': 'swift'}, {'Skills': 'Objective-C'...",5dca13caf68f858820733ca1


In [157]:
df['Education'][0]

[{'School': 'University of Washington',
  'Degree': 'Biochemistry Applied Mathematics/Chemistry minor ',
  'Date Attended': '2008 – 2013'},
 {'School': 'University of Washington', 'Degree': '  ', 'Date Attended': ''}]

In [158]:
# 불필요 열 삭제
null_list = ['_id', 'Profile Summary', 'Date Captured']

df.drop(null_list, axis = 1, inplace=True)
df.columns

Index(['Education', 'Experience', 'Industry Knowledge', 'Interpersonal Skills',
       'Job Title', 'Languages', 'Location', 'Other Skills', 'ProfileID',
       'Skills & Endorsements', 'Tools & Technologies'],
      dtype='object')

In [159]:
# 중복 제거 (ProfileID 기준)
df = df.drop_duplicates('ProfileID')

In [160]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 799 entries, 0 to 1006
Data columns (total 11 columns):
Education                799 non-null object
Experience               799 non-null object
Industry Knowledge       715 non-null object
Interpersonal Skills     276 non-null object
Job Title                799 non-null object
Languages                24 non-null object
Location                 799 non-null object
Other Skills             700 non-null object
ProfileID                799 non-null object
Skills & Endorsements    799 non-null object
Tools & Technologies     725 non-null object
dtypes: object(11)
memory usage: 74.9+ KB


In [161]:
# ProfileID 대신 ID열 생성
df['ID'] = df.index

In [162]:
# ID 테이블 생성
df_ID = pd.DataFrame(df[['ID', 'ProfileID']])
df_ID

,ID,ProfileID
0,0,jilliankim
1,1,hyunsikchoi
2,2,rui-deng-b7b37714a
3,3,seungjoonlee1984
4,4,taeho-jeon-78418870
5,5,jae-eun-jenn-choi-3487b6106
6,6,myungsoo-kim-63672462
7,7,justinlee84
8,8,yli262
9,9,qihang-yan


In [163]:
# df 에서 ProfileID 열 삭제
del df['ProfileID']
df

,Education,Experience,Industry Knowledge,Interpersonal Skills,Job Title,Languages,Location,Other Skills,Skills & Endorsements,Tools & Technologies,ID
0,"[{'School': 'University of Washington', 'Degre...","[{'Job Title': 'Senior Data Engineer', 'Compan...","[{'Skills': 'Research'}, {'Skills': 'Data Anal...",NaN,Senior Data Engineer at Change Healthcare,NaN,San Francisco Bay Area,"[{'Skills': 'Apache Pig'}, {'Skills': 'Apache ...","[{'Skills': 'Apache Spark'}, {'Skills': 'Apach...","[{'Skills': 'Java'}, {'Skills': 'HBase'}, {'Sk...",0
1,"[{'School': 'Korea University', 'Degree': 'Doc...","[{'Job Title': 'Director of Research', 'Compan...","[{'Skills': 'Algorithms'}, {'Skills': 'Big Dat...",NaN,Sr. Principal Data Engineer at Coupang Global ...,NaN,San Francisco Bay Area,"[{'Skills': 'Data Processing'}, {'Skills': 'La...","[{'Skills': 'Hadoop'}, {'Skills': 'Distributed...","[{'Skills': 'Open Source'}, {'Skills': 'Java'}...",1
2,"[{'School': 'University of Virginia', 'Degree'...","[{'Job Title': 'Data Engineer', 'Company': 'Pl...",NaN,NaN,Data Engineer at PlayStation,NaN,"Los Angeles, California",NaN,"[{'Skills': 'Python (Programming Language)'}, ...",NaN,2
3,"[{'School': 'University of California, Berkele...",[{'Job Title': 'Senior Site Reliability Engine...,"[{'Skills': 'Web Applications'}, {'Skills': 'W...",NaN,Architect / Data engineer,NaN,"Berkeley, California","[{'Skills': 'Memcached'}, {'Skills': 'Google A...","[{'Skills': 'JavaScript'}, {'Skills': 'Python'...","[{'Skills': 'Java'}, {'Skills': 'JSON'}, {'Ski...",3
4,"[{'School': 'Flatiron School', 'Degree': ' Dat...","[{'Job Title': 'Data Scientist Student', 'Comp...","[{'Skills': 'Web Development'}, {'Skills': 'An...",[{'Skills': 'Written Communication'}],Data Scientist and ML Engineer,NaN,Greater Seattle Area,"[{'Skills': '모바일앱'}, {'Skills': 'C++ Language'...","[{'Skills': 'iOS development'}, {'Skills': 'Ja...","[{'Skills': 'swift'}, {'Skills': 'Objective-C'...",4
5,"[{'School': 'Carnegie Mellon University', 'Deg...","[{'Job Title': 'Data Engineer', 'Company': 'Ae...","[{'Skills': 'Research'}, {'Skills': 'Data Anal...",NaN,"Data Engineer at Aetna, a CVS Health Company",NaN,Greater New York City Area,NaN,"[{'Skills': 'Python'}, {'Skills': 'R'}, {'Skil...",[{'Skills': 'Microsoft PowerPoint'}],5
6,"[{'School': 'University of Oregon', 'Degree': ...","[{'Job Title': 'Data Engineer', 'Company': 'Sm...","[{'Skills': 'Research'}, {'Skills': 'Patents'}]",[{'Skills': 'Teamwork'}],Data Engineer at SmartDrive Systems,NaN,Greater San Diego Area,"[{'Skills': 'Deep Learning'}, {'Skills': 'Data...","[{'Skills': 'Data Preparation'}, {'Skills': 'W...","[{'Skills': 'Hadoop'}, {'Skills': 'Python'}, {...",6
7,"[{'School': 'Sungkyunkwan University', 'Degree...","[{'Job Title': 'Company Name Coupang', 'Compan...",[{'Skills': 'Data Warehousing'}],NaN,Sr. Data Engineer at Coupang,NaN,"Seattle, Washington","[{'Skills': 'Spark'}, {'Skills': 'Teradata'}, ...","[{'Skills': 'SQL'}, {'Skills': 'Python'}, {'Sk...","[{'Skills': 'Linux'}, {'Skills': 'Java'}, {'Sk...",7
8,"[{'School': 'The Wharton School', 'Degree': 'B...","[{'Job Title': 'Summer Intern', 'Company': 'Bo...",NaN,NaN,Student at University of Pennsylvania,NaN,"Philadelphia, Pennsylvania",NaN,"[{'Skills': 'Java'}, {'Skills': 'C++'}, {'Skil...","[{'Skills': 'Css'}, {'Skills': 'Microsoft Powe...",8
9,"[{'School': 'Cornell University', 'Degree': 'B...","[{'Job Title': 'Incoming Data Engineer', 'Comp...","[{'Skills': 'Social Media'}, {'Skills': 'Proje...","[{'Skills': 'Teaching'}, {'Skills': 'Leadershi...",Incoming Data Engineer at Amazon Robotics,NaN,"Ithaca, New York","[{'Skills': 'Ad-hoc'}, {'Skills': 'Information...","[{'Skills': 'Java'}, {'Skills': 'Python'}, {'S...","[{'Skills': 'C'}, {'Skills': 'Microsoft Excel'...",9


In [187]:
# 인덱스 재 설정 (중복행 제거로 뒤섞임)
df = df.reset_index(drop=True)
df.index

RangeIndex(start=0, stop=799, step=1)

In [188]:
df_copy = df.copy()

### df 통째로 lower 하는 방법?

In [166]:
# columns = ['Education', 'Experience', 'Industry Knowledge',
#        'Interpersonal Skills', 'Job Title', 'Languages', 'Location',
#        'Other Skills', 'Skills & Endorsements',
#        'Tools & Technologies']

# for col in columns:
#     df[col] = df[col].str.lower() 
#     df.head()

In [200]:
# Job_list 소문자로 변경
job_list = []

for job in df['Job Title']:
    job_list.append(job.lower())

job_list

['senior data engineer at change healthcare',
 'sr. principal data engineer at coupang global llc.',
 'data engineer at playstation',
 'architect / data engineer',
 'data scientist and ml engineer',
 'data engineer at aetna, a cvs health company',
 'data engineer at smartdrive systems',
 'sr. data engineer at coupang',
 'incoming data engineer at amazon robotics',
 'data engineer at united technologies',
 'data engineer at google',
 'data engineer at facebook',
 'data engineer | big data | hadoop | etl | informatica | data & analytics',
 'data engineer at coupang',
 'data engineer at memebox corporation',
 'data engineer intern at seattle seahawks',
 'data engineer at pfj',
 'senior data engineer - salesforce einstein',
 'data engineer at microsoft',
 'big data etl developer at doubleverify',
 'data engineer ii at google',
 'data engineer @ one medical',
 'data scientist / research engineer at microsoft',
 'senior data engineer',
 'data engineer at robinhood',
 'data engineer',
 'data 

In [191]:
len(job_list)

799

In [192]:
de_list = ['engineer', 'platform', 'infrastructure', 'science', 'scientist', 'big', 'specialist', 'operations']

In [201]:
# test ; data랑 engineer가 포함 된 직업 탐색
test = []
for i, job in enumerate(job_list):
    if 'data' and 'engineer' not in job:
        test.append(i)
        
len(test)

46

# Job Title Nomalization

In [202]:
check=[]
del_list = []

for i, job in enumerate(job_list):
    for de in de_list:
        check.append(de in job)
#     print(check)
    if 'data' in job and any(check):
        pass
    else:
        del_list.append(i)
    del check[:]
print(del_list)
len(del_list)

[]


0

In [195]:
for i in del_list:
    print(i, job_list[i])

8 student at university of pennsylvania
13 i'm student
16 events directo at data visualization society
23 staff associate at columbia university irving medical center
24 software engineer
156 
157 
158 
159 
174 software engineer at alatting
313 data@netflix
317 applied research engineer
344 software engineer at facebook
345 software engineer at microsoft
366 
367 
368 
369 
370 
371 
372 
377 software developer at lyft
405 software engineer at goldman sachs asset management
435 
504 general engineer at national reconnaissance office (nro)
516 engineer at gracenote
520 senior machine learning engineer at wework
545 
560 actively seeking full time opportunities | cmu grad | ex- google, jp morgan chase
561 
563 senior manager, data
571 data analyst at boeing
577 data analytics professional
601 
605 data analytics in fintech
607 
613 data at lyft
626 data analyst @ google | actively looking for full-time opportunities
629 cmu grad | ghc'19 | paypal | jp morgan chase
632 software engineer 

In [196]:
df.iloc[366]

Education                 []
Experience                []
Industry Knowledge       NaN
Interpersonal Skills     NaN
Job Title                   
Languages                NaN
Location                    
Other Skills             NaN
Skills & Endorsements     []
Tools & Technologies     NaN
ID                       366
Name: 366, dtype: object

In [199]:
# del_list 기준으로 df 에서 삭제
df.drop(del_list, inplace=True)
len(df)

733

In [206]:
# 남은 Job_Title값을 전부 'data engineer' 로 변경
df['Job Title'] = 'data engineer'
df['Job Title'].unique()

array(['data engineer'], dtype=object)

# Education Nomalization

In [248]:
# 인덱스 재 설정 (중복행 제거로 뒤섞임)
df = df.reset_index(drop=True)
df.index

RangeIndex(start=0, stop=733, step=1)

In [249]:
df['Education'][0][0]

{'School': 'University of Washington',
 'Degree': 'Biochemistry Applied Mathematics/Chemistry minor ',
 'Date Attended': '2008 – 2013'}

In [250]:
df['Education']

0      [{'School': 'University of Washington', 'Degre...
1      [{'School': 'Korea University', 'Degree': 'Doc...
2      [{'School': 'University of Virginia', 'Degree'...
3      [{'School': 'University of California, Berkele...
4      [{'School': 'Flatiron School', 'Degree': ' Dat...
5      [{'School': 'Carnegie Mellon University', 'Deg...
6      [{'School': 'University of Oregon', 'Degree': ...
7      [{'School': 'Sungkyunkwan University', 'Degree...
8      [{'School': 'Cornell University', 'Degree': 'B...
9      [{'School': 'Western University', 'Degree': 'M...
10     [{'School': 'Barnard College', 'Degree': 'Bach...
11     [{'School': 'University of California, Berkele...
12     [{'School': 'Western Illinois University', 'De...
13     [{'School': 'Soongsil University', 'Degree': '...
14     [{'School': 'Konkuk University', 'Degree': 'Un...
15     [{'School': 'University of Washington', 'Degre...
16     [{'School': 'William & Mary', 'Degree': 'Maste...
17     [{'School': 'Carnegie Me

In [299]:
# Education key 값 확인.
edu_key = []

for i in range(len(df)):
    if df['Education'][i] == []:
        pass
    else:
        for k in df['Education'][i][0].keys():
            edu_key.append(k)
        
set(edu_key)

{'Date Attended', 'Degree', 'School'}

In [335]:
df['School'] = None
df['Degree'] = None
df['Date Attended'] = None
df.head()

,Education,Experience,Industry Knowledge,Interpersonal Skills,Job Title,Languages,Location,Other Skills,Skills & Endorsements,Tools & Technologies,ID,School,Degree,Date Attended
0,"[{'School': 'University of Washington', 'Degre...","[{'Job Title': 'Senior Data Engineer', 'Compan...","[{'Skills': 'Research'}, {'Skills': 'Data Anal...",NaN,data engineer,NaN,San Francisco Bay Area,"[{'Skills': 'Apache Pig'}, {'Skills': 'Apache ...","[{'Skills': 'Apache Spark'}, {'Skills': 'Apach...","[{'Skills': 'Java'}, {'Skills': 'HBase'}, {'Sk...",0,None,None,None
1,"[{'School': 'Korea University', 'Degree': 'Doc...","[{'Job Title': 'Director of Research', 'Compan...","[{'Skills': 'Algorithms'}, {'Skills': 'Big Dat...",NaN,data engineer,NaN,San Francisco Bay Area,"[{'Skills': 'Data Processing'}, {'Skills': 'La...","[{'Skills': 'Hadoop'}, {'Skills': 'Distributed...","[{'Skills': 'Open Source'}, {'Skills': 'Java'}...",1,None,None,None
2,"[{'School': 'University of Virginia', 'Degree'...","[{'Job Title': 'Data Engineer', 'Company': 'Pl...",NaN,NaN,data engineer,NaN,"Los Angeles, California",NaN,"[{'Skills': 'Python (Programming Language)'}, ...",NaN,2,None,None,None
3,"[{'School': 'University of California, Berkele...",[{'Job Title': 'Senior Site Reliability Engine...,"[{'Skills': 'Web Applications'}, {'Skills': 'W...",NaN,data engineer,NaN,"Berkeley, California","[{'Skills': 'Memcached'}, {'Skills': 'Google A...","[{'Skills': 'JavaScript'}, {'Skills': 'Python'...","[{'Skills': 'Java'}, {'Skills': 'JSON'}, {'Ski...",3,None,None,None
4,"[{'School': 'Flatiron School', 'Degree': ' Dat...","[{'Job Title': 'Data Scientist Student', 'Comp...","[{'Skills': 'Web Development'}, {'Skills': 'An...",[{'Skills': 'Written Communication'}],data engineer,NaN,Greater Seattle Area,"[{'Skills': '모바일앱'}, {'Skills': 'C++ Language'...","[{'Skills': 'iOS development'}, {'Skills': 'Ja...","[{'Skills': 'swift'}, {'Skills': 'Objective-C'...",4,None,None,None


In [336]:
for i in range(len(df)):
    if df['Education'][i] == []:
        pass
    else:
        df['School'][i] = df['Education'][i][0]['School']
        df['Degree'][i] = df['Education'][i][0]['Degree']
        df['Date Attended'][i] = df['Education'][i][0]['Date Attended']

In [ ]:
del df['Education']

In [340]:
df.to_csv('cleansing_191116')